In [ ]:
!pip install thop
!pip install onnxruntime
!pip install onnxruntime-gpu
!pip install ultralytics

In [2]:
import gc
import os
import psutil
import shutil
import time
import cv2
import numpy as np
import torch
import onnxruntime as ort
from google.colab import drive
from copy import deepcopy
from ultralytics import YOLO
from thop import profile


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
drive.mount('/content/gdrive/')

!unzip -q /content/gdrive/MyDrive/data.zip

Mounted at /content/gdrive/


In [4]:
def get_memory_usage():
    """Получение текущего потребления памяти"""
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024  # в МБ

def run_inference_pytorch(model, images, device='cpu'):
    """Запуск инференса на PyTorch модели"""
    model.model.to(device)
    times = []

    for img_name in images:
        img_path = os.path.join(test_folder, img_name)
        start_time = time.time()

        results = model(img_path, verbose=False)

        end_time = time.time()
        times.append(end_time - start_time)

    return times

def run_inference_onnx(model_path, images, device='cpu'):
    """Запуск инференса на ONNX модели"""
    providers = ['CPUExecutionProvider'] if device == 'cpu' else ['CUDAExecutionProvider']
    session = ort.InferenceSession(model_path, providers=providers)
    times = []

    for img_name in images:
        img_path = os.path.join(test_folder, img_name)

        # Подготовка входных данных для ONNX
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (640, 640))
        img = img.astype(np.float32) / 255.0
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0)

        input_name = session.get_inputs()[0].name

        start_time = time.time()

        outputs = session.run(None, {input_name: img})

        end_time = time.time()
        times.append(end_time - start_time)

    return times


In [5]:
# Пути

test_folder = './test'
colab_output_folder = './test_results'
drive_output_folder = '/content/gdrive/MyDrive/test_results'

In [6]:
test_images = [f for f in os.listdir(test_folder) if f.lower().endswith('.jpg')]
test_count = len(test_images)

print(f"Количество фото в папке test: {test_count}")

Количество фото в папке test: 81


In [7]:
# Загрузка моделей

pytorch_model = YOLO('best_model.pt')
onnx_model = 'best_model.onnx'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


#### Измерения на GPU

In [8]:
# PyTorch

torch.cuda.empty_cache()
gc.collect()
start_time = time.time()

pytorch_gpu_times = run_inference_pytorch(pytorch_model, test_images, 'cuda')
pytorch_gpu_total_time = time.time() - start_time

print(f"PyTorch - Общее время обработки: {pytorch_gpu_total_time:.2f} сек")
print(f"PyTorch - Среднее время на снимок: {np.mean(pytorch_gpu_times):.4f} сек")


PyTorch - Общее время обработки: 23.68 сек
PyTorch - Среднее время на снимок: 0.2893 сек


In [9]:
# ONNX

torch.cuda.empty_cache()
gc.collect()
start_time = time.time()

onnx_gpu_times = run_inference_onnx(onnx_model, test_images, 'cuda')
onnx_gpu_total_time = sum(onnx_gpu_times)

print(f"ONNX - Общее время обработки: {onnx_gpu_total_time:.2f} сек")
print(f"ONNX - Среднее время на снимок: {np.mean(onnx_gpu_times):.4f} сек")


ONNX - Общее время обработки: 3.64 сек
ONNX - Среднее время на снимок: 0.0449 сек
